Задание.
п1. Загрузите данные. Используйте датасет с ирисами. В данных оставьте только 2 класса: Iris Versicolor, Iris Virginica.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score, precision_score, f1_score
from sklearn.model_selection import train_test_split
from datetime import datetime

In [ ]:
data = load_iris()

In [ ]:
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target
df_iris = df[(df['target'] == 1) | (df['target'] == 2)]
df_iris.loc[df_iris['target'] == 1, 'target'] = 0
df_iris.loc[df_iris['target'] == 2, 'target'] = 1
df_iris

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
50,7.0,3.2,4.7,1.4,0
51,6.4,3.2,4.5,1.5,0
52,6.9,3.1,4.9,1.5,0
53,5.5,2.3,4.0,1.3,0
54,6.5,2.8,4.6,1.5,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,1
146,6.3,2.5,5.0,1.9,1
147,6.5,3.0,5.2,2.0,1
148,6.2,3.4,5.4,2.3,1


В датасете значения столбца 'target' соответствует 1 и 2 для Iris-versicolor и Iris-virginic соответственно. Поэтому оставляем только эти категории, затем переобозначаем значения в 0 и 1.

Разделяем датасет на признаки X и целевую переменную с категориями y.

In [ ]:
X = df_iris.drop(columns='target')
y = df_iris['target']

п2. Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки.

Для реализации логистической регрессии были написана функция, предсказывающая, к какой категории ирисов относится каждый образец.

Вероятность принадлежности к классу при логистической регрессии рассчитывается как сигмоида от суммы произведений значения признака на вес этого признака плюс свободный член. Затем, в зависимости от полученной вероятности, делается вывод о принадлежности каждого ириса к конкретному классу.

In [ ]:
def sigmoid(z): #функция для расчета сигмоиды
  return 1/(1 + np.exp(-z))

In [ ]:
def sigm_der(z): #производная сигмоиды
  return sigmoid(z) * (1 - sigmoid(z))

In [ ]:
def log_regression(X, w, w0): #функция для логистической регрессии
  class_pred = []
  X = np.array(X)
  for x in X:
    z = w0 + w[0]*x[0] + w[1]*x[1] + w[2]*x[2] + w[3]*x[3] #сумма произведений вектора весов и вектора признаков + w0
    class_pred.append(1 if sigmoid(z) >= 0.5 else 0) #считаем, что класс 1, если полученная вероятность >= 0.5
  return class_pred

п3. Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. Выберете и посчитайте метрику качества.

При помощи метода градиентного спуска определяем значения весов параметров.

In [ ]:
def cost_function(w, w0, X, y):
  cost = 0
  for j in range(len(X)):
    z = w0 + w[0]*X[0] + w[1]*X[1] + w[2]*X[2] + w[3]*X[3]
    cost -= y[j] * np.log(sigmoid(z)) + (1-y[j])*np.log(1-sigmoid(z)) #logloss
    #print(cost)
  return cost/len(X)

In [ ]:
def grad_des(X, y, learning_rate=0.001, epochs=100):
  y = np.array(y)
  X = np.array(X)
  w = np.zeros(4)
  w0 = 0
  costs = []
  for p in range(epochs):
    y_pred = []
    for i in range(len(X)):
      z = w0 + w[0]*X[i][0] + w[1]*X[i][1] + w[2]*X[i][2] + w[3]*X[i][3]
      y_pred.append(sigmoid(z))
      w0 -= learning_rate * (y_pred[i] - y[i]) * sigm_der(y_pred[i])
      for k in range(4):
        w[k] -= learning_rate * (y_pred[i] - y[i]) * X[i][k] * sigm_der(y_pred[i]) #считаем веса по методу градиентного спуска
    cost = cost_function(w, w0, X, y) #считаем ошибки в весах в каждую эпоху
    costs.append(cost)
  return(w0, w, costs)

Проверяем работу функций. Определяем значения весов параметров и обучаем модель предсказывать, к какому классу относится каждый ирис.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) #разделение на тренировочную и тестовую выборки

In [ ]:
start_time = datetime.now()
w0, w, costs = grad_des(X_train, y_train, learning_rate=0.001, epochs=500) #вычисляем веса параметров модели и ошибку
print(f'Полученные методом градиентного спуска веса параметров: {w0, w}')
print(f'Последнее значение ошибки параметров {costs[-1]}')
class_pred = log_regression(X_test, w, w0) #подставляем веса и предсказываем значения класса на тестовой выборке
print(f'Предсказанные значения классов на тренировочной выборке: {class_pred}')
end_time = datetime.now()
work_time_grad = (end_time - start_time).total_seconds()

Полученные методом градиентного спуска веса параметров: (-0.29564850028283246, array([-0.62898273, -0.58125431,  0.96574638,  0.79794051]))
Последнее значение ошибки параметров [1.31953735 0.76003273 0.72133896 0.71749017]
Предсказанные значения классов на тренировочной выборке: [1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0]


В качестве метрики точности была выбрана метрика F1-score, так как она представляет собой среднее точности и полноты.

In [ ]:
print(f'Значение F1-score: {f1_score(y_test, class_pred)}')

Значение F1-score: 0.8421052631578948


Итого в результате обучения модели логистической регрессии методом градиентного спуска на тестовой выборке получили значение F1-score = 0.84. Результат показывает, что модель достаточно хорошо обучена.

п4. Повторите п. 3 для метода скользящего среднего (Root Mean Square Propagation, RMSProp).

Формулы были взяты по ссылке: https://habr.com/en/articles/318970/

In [ ]:
def RMSProp(X, y, learning_rate=0.001, epochs=100, gamma = 0.9, epsilon=1e-8):
  y = np.array(y)
  X = np.array(X)
  w = np.zeros(4)
  w0 = 0
  rms_w0 = 0
  rms_w = np.zeros(4)
  costs =[]
  for p in range(epochs):
    y_pred = []
    for i in range(len(X)):
      z = w0 + w[0]*X[i][0] + w[1]*X[i][1] + w[2]*X[i][2] + w[3]*X[i][3]
      y_pred.append(sigmoid(z))
      rms_w0 = gamma * rms_w0 + (1 - gamma) * ((y_pred[i] - y[i]) * sigm_der(y_pred[i]))**2
      for k in range(X.shape[1]):
        rms_w[k] = gamma * rms_w[k] + (1 - gamma) * ((y_pred[i] - y[i]) * X[i][k] * sigm_der(y_pred[i]))**2
      w0 -= (learning_rate * (y_pred[i] - y[i]) * sigm_der(y_pred[i])) / np.sqrt(rms_w0 + epsilon)
      for k in range(X.shape[1]):
        w[k] -= (learning_rate * (y_pred[i] - y[i]) * X[i][k] * sigm_der(y_pred[i])) / np.sqrt(rms_w[k] + epsilon)
    cost = cost_function(w, w0, X, y)
    costs.append(cost)
  return(w0, w, costs)

In [ ]:
start_time1 = datetime.now()
w0, w, costs = RMSProp(X_train, y_train, learning_rate=0.001, epochs=500, gamma = 0.9, epsilon=1e-8)
print(f'Полученные методом скользящего среднего веса параметров: {w0, w}')
print(f'Последнее значение ошибки параметров {costs[-1]}')
class_pred_rms = log_regression(X_test, w, w0)
print(f'Предсказанные значения классов на тестовой выборке: {class_pred_rms}')
end_time1 = datetime.now()
work_time_rms = (end_time1 - start_time1).total_seconds()

Полученные методом скользящего среднего веса параметров: (-2.368182530150962, array([-0.88837538, -1.74490305,  1.48867355,  3.47941106]))
Последнее значение ошибки параметров [5.08931975 1.45087854 2.12827284 0.86902977]
Предсказанные значения классов на тестовой выборке: [0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0]


In [ ]:
print(f'Значение F1-score: {f1_score(y_test, class_pred_rms)}')

Значение F1-score: 0.7777777777777777


Итого в результате обучения модели логистической регрессии методом скользящего среднего на тестовой выборке получили значение F1-score = 0.78.



п6. Сравните значение метрик для реализованных методов оптимизации. Можно оформить в виде таблицы вида |метод|метрика|время работы| (время работы опционально).

In [ ]:
result = pd.DataFrame({'Метод': ['Градиентный спуск', 'Скользящее среднее'], 'Метрика': [f1_score(y_test, class_pred), f1_score(y_test, class_pred_rms)], 'Время работы, сек': [work_time_grad, work_time_rms]})
result

,Метод,Метрика,"Время работы, сек"
0,Градиентный спуск,0.842105,2.425185
1,Скользящее среднее,0.777778,4.416313


В результате выполнения работы была построена модель логистической регрессии, обученная методами градиентного спуска и скользящего среднего.

Из таблицы следует, что модель логистической регресии, обученная методом градиентного спуска, имеет метрику качества выше, т.е. точнее предсказывает значения класса ирисов. Также время работы в данном случае в 2 раза быстрее, чем при работе методом с скользящего среднего.